#need to test the code

In [1]:
!pip install Pillow cryptography


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: C:\Users\ray81\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [ ]:
import cv2
import numpy as np

def add_watermark(image_path, watermark_text, output_path):
    # Load the original image
    image = cv2.imread(image_path)

    # Convert image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create a binary watermark image
    watermark = np.zeros_like(gray)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(watermark, watermark_text, (10, 50), font, 1, 255, 2, cv2.LINE_AA)

    # Embed the watermark by adding it to the LSB of the image
    watermarked = cv2.addWeighted(gray, 1, watermark, 0.1, 0)
    watermarked = cv2.merge([watermarked, watermarked, watermarked])

    # Save the watermarked image
    cv2.imwrite(output_path, watermarked)

# Example usage
add_watermark('photo.jpg', 'My Watermark', 'watermarked_photo.jpg')


In [2]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives.asymmetric.utils import Prehashed

# Generate a private key for signing
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)

# Save the private key
with open("private_key.pem", "wb") as key_file:
    key_file.write(
        private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        )
    )

# Generate a public key for verification
public_key = private_key.public_key()

# Save the public key
with open("public_key.pem", "wb") as key_file:
    key_file.write(
        public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        )
    )

# Function to create a digital signature
def create_signature(image_path, private_key):
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    # Create a hash of the image
    image_hash = hashes.Hash(hashes.SHA256())
    image_hash.update(image_data)
    digest = image_hash.finalize()

    # Sign the hash with the private key
    signature = private_key.sign(
        digest,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        Prehashed(hashes.SHA256())
    )

    return signature

# Create a digital signature
signature = create_signature("photo.jpg", private_key)

# Save the signature to a file
with open("signature.bin", "wb") as signature_file:
    signature_file.write(signature)


FileNotFoundError: [Errno 2] No such file or directory: 'photo.jpg'

In [3]:
#vertify digital signature

from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes, serialization

# Function to verify a digital signature
def verify_signature(image_path, signature, public_key):
    with open(image_path, "rb") as image_file:
        image_data = image_file.read()

    # Create a hash of the image
    image_hash = hashes.Hash(hashes.SHA256())
    image_hash.update(image_data)
    digest = image_hash.finalize()

    # Verify the signature with the public key
    try:
        public_key.verify(
            signature,
            digest,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            Prehashed(hashes.SHA256())
        )
        print("Signature is valid.")
    except Exception as e:
        print("Signature verification failed:", str(e))

# Load the public key
with open("public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(key_file.read())

# Load the signature
with open("signature.bin", "rb") as signature_file:
    signature = signature_file.read()

# Verify the signature
verify_signature("photo.jpg", signature, public_key)


FileNotFoundError: [Errno 2] No such file or directory: 'signature.bin'

from PIL import Image, ImageDraw, ImageFont

def add_signature(image_path, signature_text, output_path):
    # Open an image file
    with Image.open(image_path) as img:
        draw = ImageDraw.Draw(img)
        font = ImageFont.truetype("arial.ttf", 50)  # Adjust font and size as needed

        # Position for the signature (bottom right corner)
        text_width, text_height = draw.textsize(signature_text, font)
        width, height = img.size
        x = width - text_width - 10
        y = height - text_height - 10

        # Add the signature to the image
        draw.text((x, y), signature_text, font=font, fill=(255, 255, 255))

        # Save the image with the signature
        img.save(output_path)

# Example usage
add_signature("photo.jpg", "John Doe", "signed_photo.jpg")


In [ ]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.exceptions import InvalidSignature

# Load the public key
with open("public_key.pem", "rb") as key_file:
    public_key = serialization.load_pem_public_key(key_file.read())

# Load the signature
with open("signature.sig", "rb") as signature_file:
    signature = signature_file.read()

# Load the photo
with open("photo.jpg", "rb") as photo_file:
    photo_data = photo_file.read()

# Verify the signature
try:
    public_key.verify(
        signature,
        photo_data,
        padding.PKCS1v15(),
        hashes.SHA256()
    )
    print("Signature is valid.")
except InvalidSignature:
    print("Signature is invalid.")


In [ ]:
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import padding

# Load the private key
with open("private_key.pem", "rb") as key_file:
    private_key = serialization.load_pem_private_key(
        key_file.read(),
        password=None  # or provide the password if the key is encrypted
    )

# Load the photo
with open("photo.jpg", "rb") as photo_file:
    photo_data = photo_file.read()

# Create the hash and sign it
signature = private_key.sign(
    photo_data,
    padding.PKCS1v15(),
    hashes.SHA256()
)

# Save the signature to a file
with open("signature.sig", "wb") as signature_file:
    signature_file.write(signature)


In [ ]:
import cv2
import numpy as np

def embed_watermark(image_path, watermark_text, output_path):
    # Load the original image
    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    # Convert the image to float32 for better precision
    image_float = np.float32(image)
    
    # Create a binary watermark image
    watermark = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(watermark, watermark_text, (10, 50), font, 1, 255, 2, cv2.LINE_AA)
    
    # Normalize the watermark to range between 0 and 1
    watermark = watermark / 255.0
    
    # Embed the watermark in the least significant bits of the image
    watermarked_image = image_float
    watermarked_image[:,:,0] = image_float[:,:,0] * (1 - watermark) + watermark * 255
    
    # Convert back to uint8
    watermarked_image = np.uint8(watermarked_image)
    
    # Save the watermarked image
    cv2.imwrite(output_path, watermarked_image)

# Example usage
embed_watermark('photo.jpg', 'My Watermark', 'watermarked_photo.png')


In [ ]:
def extract_watermark(watermarked_image_path):
    # Load the watermarked image
    watermarked_image = cv2.imread(watermarked_image_path, cv2.IMREAD_COLOR)
    
    # Convert the image to float32
    watermarked_image_float = np.float32(watermarked_image)
    
    # Extract the watermark from the least significant bits
    extracted_watermark = watermarked_image_float[:,:,0] - np.round(watermarked_image_float[:,:,0])
    
    # Normalize the watermark to range between 0 and 255
    extracted_watermark = np.abs(extracted_watermark * 255)
    
    # Convert to uint8
    extracted_watermark = np.uint8(extracted_watermark)
    
    return extracted_watermark

# Example usage
extracted_watermark = extract_watermark('watermarked_photo.png')
cv2.imshow('Extracted Watermark', extracted_watermark)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import hashlib

def create_hash(image_path):
    # Read the image in binary mode
    with open(image_path, 'rb') as image_file:
        image_data = image_file.read()
        
    # Create a SHA-256 hash of the image
    hash_object = hashlib.sha256(image_data)
    hash_value = hash_object.hexdigest()
    
    return hash_value

# Example usage
hash_value = create_hash('photo.jpg')
print("Hash Value:", hash_value)


In [ ]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.hazmat.primitives import serialization

# Generate a private key (for demonstration purposes; usually, you have an existing key)
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)

# Serialize the private key to store it securely
private_pem = private_key.private_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PrivateFormat.TraditionalOpenSSL,
    encryption_algorithm=serialization.NoEncryption()
)

# Load the private key (in practice, load it from a secure storage)
private_key = serialization.load_pem_private_key(
    private_pem,
    password=None,
)

# Function to sign the hash
def sign_hash(hash_value, private_key):
    # Convert the hash value to bytes
    hash_bytes = bytes(hash_value, 'utf-8')
    
    # Sign the hash using the private key
    signature = private_key.sign(
        hash_bytes,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    
    return signature

# Example usage
signature = sign_hash(hash_value, private_key)
print("Digital Signature:", signature)


In [ ]:
from cryptography.hazmat.primitives.asymmetric import padding

# Serialize the public key to distribute it securely
public_key = private_key.public_key()
public_pem = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)

# Load the public key (in practice, load it from a distributed key)
public_key = serialization.load_pem_public_key(public_pem)

# Function to verify the signature
def verify_signature(image_path, signature, public_key):
    # Create the hash of the received image
    received_hash_value = create_hash(image_path)
    received_hash_bytes = bytes(received_hash_value, 'utf-8')
    
    # Verify the signature using the public key
    try:
        public_key.verify(
            signature,
            received_hash_bytes,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.MAX_LENGTH
            ),
            hashes.SHA256()
        )
        return True
    except Exception as e:
        print("Verification failed:", e)
        return False

# Example usage
verification_result = verify_signature('photo.jpg', signature, public_key)
print("Verification Result:", verification_result)


In [ ]:
from PIL import Image

def pixelate_image(input_image_path, output_image_path, pixel_size):
    # Open the input image
    image = Image.open(input_image_path)
    
    # Get the size of the image
    width, height = image.size
    
    # Resize the image to a smaller size
    small_image = image.resize(
        (width // pixel_size, height // pixel_size),
        resample=Image.NEAREST
    )
    
    # Scale it back up to the original size
    pixelated_image = small_image.resize(
        (width, height),
        Image.NEAREST
    )
    
    # Save the pixelated image
    pixelated_image.save(output_image_path)
    
# Example usage
input_image_path = 'input.jpg'
output_image_path = 'output_pixelated.jpg'
pixel_size = 10  # Adjust the pixel size for more or less pixelation

pixelate_image(input_image_path, output_image_path, pixel_size)
